In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset , DataLoader

import optuna
import mlflow
import dagshub

In [4]:
dagshub.init(repo_owner='Bilal-ahmad8', repo_name='Advanced-Monitoring-System', mlflow=True)

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

Output()



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=8dad9ea4-129c-4c68-a99e-48ab5c45ade5&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=a19027fe7dd7a9cf76bdd680085e2c78b8a1e0b269670e7d32d62d1609da68a2




Accessing as Bilal-ahmad8

Initialized MLflow to track repo "Bilal-ahmad8/Advanced-Monitoring-System"

Repository Bilal-ahmad8/Advanced-Monitoring-System initialized!

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [6]:
torch.manual_seed(42)

In [7]:
train_data = pd.read_csv('Training_df.csv')
val_data = pd.read_csv('Validation_df.csv')

In [8]:
train_data.head()

,timestamp,cpu_usage,memory_available,disk_read,disk_write,network_receive,network_transmit
0,2025-06-18 08:48:00,0.724931,0.746927,0.802243,0.010949,0.018548,0.063189
1,2025-06-18 08:48:30,0.247370,0.731033,1.000000,0.011040,0.014776,0.050499
2,2025-06-18 08:49:00,0.285406,0.729110,0.367237,0.002847,0.000205,0.000320
3,2025-06-18 08:49:30,0.438017,0.738016,0.021358,0.001111,0.000761,0.003752
4,2025-06-18 08:50:00,0.214782,0.745808,0.000000,0.001976,0.000601,0.000559


In [9]:
def create_sequences(data, seq_len):
  sequences = []
  for i in range(len(data)-seq_len+1):
    seq = data[i:i+seq_len]
    sequences.append(seq)
  sequences = torch.tensor(sequences, dtype = torch.float32)

  return sequences, sequences.clone()


In [10]:
class CustomDataset(Dataset):
  def __init__(self,data, seq_len):
    self.data , self.targets = create_sequences(data, seq_len)

  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx):
    return self.data[idx], self.targets[idx]

In [11]:
train_dataset = CustomDataset(train_data.drop(['timestamp'], axis=1).values,20)
val_dataset = CustomDataset(val_data.drop(['timestamp'], axis=1).values, 20)

/tmp/ipython-input-9-2462898161.py:6: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:254.)
  sequences = torch.tensor(sequences, dtype = torch.float32)


In [12]:
print(len(train_dataset))
print(len(val_dataset))

251
52


In [13]:
class LSTMAutoEncoder(nn.Module):
  def __init__(self, input_dim, hidden_dim, num_layers, biDirect_bool, dp_ratio_1, dp_ratio_2,inside_dp_ratio, latent_dim):
    super().__init__()
    self.encoder = nn.LSTM(input_size=input_dim, hidden_size=hidden_dim, num_layers=num_layers,
                           dropout=inside_dp_ratio,batch_first=True, bidirectional=biDirect_bool)

    self.dropout = nn.Dropout(dp_ratio_1)
    self.fc = nn.Linear(hidden_dim*2 if biDirect_bool else hidden_dim, latent_dim)

    self.decoder = nn.LSTM(input_size=latent_dim, hidden_size=hidden_dim, num_layers=num_layers,
                           dropout=inside_dp_ratio, batch_first=True)

    self.dropout2 = nn.Dropout(dp_ratio_2)
    self.fc2 = nn.Linear(hidden_dim, input_dim)

  def forward(self, x):
    _, (h_n, _) = self.encoder(x)

    if self.encoder.bidirectional:
      h_n_combined = torch.cat((h_n[-2], h_n[-1]), dim=1)
    else:
      h_n_combined = h_n[-1]

    h_n_combined = self.dropout(h_n_combined)
    fc1_out = self.fc(h_n_combined)
    latent = fc1_out.unsqueeze(1).repeat(1, x.size(1), 1)

    decoded, _ = self.decoder(latent)
    decoded = self.dropout2(decoded)
    final_decoded = self.fc2(decoded)

    return final_decoded


In [14]:
def train(model, trainloader, batch_size,optimizer, criterion):
  model.train()
  total_loss = 0.0
  for data , target in trainloader:
    data, target = data.to(device), target.to(device)
    optimizer.zero_grad()
    output = model(data)
    loss = criterion(output, target)
    loss.backward()
    optimizer.step()

    total_loss += loss.item()

  return total_loss/ len(trainloader)


In [15]:
def val(model, valloader, batch_size,optimizer, criterion):
  model.eval()
  total_loss = 0.0
  with torch.no_grad():
    for data , target in valloader:
      data, target = data.to(device), target.to(device)
      output = model(data)
      loss = criterion(output, target)

      total_loss += loss.item()

  return total_loss/ len(valloader)

In [ ]:
def objective(trial):
  epochs = trial.suggest_int('epochs', 10, 100, step=10)

  lr = trial.suggest_float('lr', 1e-4, 1e-1, log=True)
  weight_decay = trial.suggest_float('weight_decay', 1e-4, 1e-1, log=True)

  hidden_dim = trial.suggest_int('hidden_dim', 32, 256, step=32)
  num_layers = trial.suggest_int('num_layers', 1, 3)
  biDirect_bool = trial.suggest_categorical('biDirect_bool',[True, False])
  dp_ratio_1 = trial.suggest_float('dp_ratio_1', 0.0, 0.5, step=0.1)
  dp_ratio_2 = trial.suggest_float('dp_ratio_2', 0.0, 0.5, step=0.1)
  inside_dp_ratio = trial.suggest_float('inside_dp_ratio', 0.0, 0.5, step=0.1)
  latent_dim = trial.suggest_int('latent_dim', 16, 32, step=8)

  batch_size = trial.suggest_categorical('batch_size', [8, 16])
  INPUT_DIM = 6
  input_dim = INPUT_DIM

  with mlflow.start_run(nested=True):
    mlflow.log_params({'epochs':epochs,
                       'lr': lr,
                       'weight_decay': weight_decay,
                       'hidden_dim': hidden_dim,
                       'num_layers': num_layers,
                       'biDirect_bool': biDirect_bool,
                       'dp_ratio_1': dp_ratio_1,
                       'dp_ratio_2': dp_ratio_2,
                       'inside_dp_ratio': inside_dp_ratio,
                       'latent_dim': latent_dim,
                       'batch_size': batch_size})

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False,pin_memory=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, pin_memory=True)
    model = LSTMAutoEncoder(input_dim, hidden_dim, num_layers, biDirect_bool, dp_ratio_1, dp_ratio_2,inside_dp_ratio, latent_dim).to(device)

    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    criterion = nn.MSELoss()

    for i in range(epochs):
      train_loss = train(model, train_loader, batch_size, optimizer, criterion)
    trial.set_user_attr('train_loss_per_epoch', train_loss)

    mlflow.log_metric("Train_loss", train_loss)

    val_loss = val(model, val_loader, batch_size, optimizer, criterion)
    trial.set_user_attr('val_loss_per_epoch', val_loss)
    mlflow.log_metric('Val_loss', val_loss)


    return val_loss



In [ ]:
study = optuna.create_study(direction='minimize')

[I 2025-06-23 09:50:43,939] A new study created in memory with name: no-name-3e59c048-cc06-463b-b819-3635ff44c846


In [ ]:
study.optimize(objective, n_trials=60)

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.30000000000000004 and num_layers=1
  warnings.warn(
[I 2025-06-23 09:50:47,472] Trial 0 finished with value: 0.01926674065180123 and parameters: {'epochs': 20, 'lr': 0.008091139267476488, 'weight_decay': 0.001150780155132052, 'hidden_dim': 192, 'num_layers': 1, 'biDirect_bool': True, 'dp_ratio_1': 0.5, 'dp_ratio_2': 0.5, 'inside_dp_ratio': 0.30000000000000004, 'latent_dim': 16, 'batch_size': 16}. Best is trial 0 with value: 0.01926674065180123.


🏃 View run gregarious-koi-854 at: https://dagshub.com/Bilal-ahmad8/Advanced-Monitoring-System.mlflow/#/experiments/0/runs/38fd95b11bb447b7b41d719abb6a0e49
🧪 View experiment at: https://dagshub.com/Bilal-ahmad8/Advanced-Monitoring-System.mlflow/#/experiments/0


[I 2025-06-23 09:51:19,573] Trial 1 finished with value: 0.022453721213553633 and parameters: {'epochs': 80, 'lr': 0.003523892048455717, 'weight_decay': 0.00011877052500408, 'hidden_dim': 224, 'num_layers': 3, 'biDirect_bool': True, 'dp_ratio_1': 0.5, 'dp_ratio_2': 0.30000000000000004, 'inside_dp_ratio': 0.30000000000000004, 'latent_dim': 32, 'batch_size': 8}. Best is trial 0 with value: 0.01926674065180123.


🏃 View run gaudy-perch-227 at: https://dagshub.com/Bilal-ahmad8/Advanced-Monitoring-System.mlflow/#/experiments/0/runs/65abe81f5c1143d8aadcc48c5ee2ca2f
🧪 View experiment at: https://dagshub.com/Bilal-ahmad8/Advanced-Monitoring-System.mlflow/#/experiments/0
🏃 View run bustling-cow-169 at: https://dagshub.com/Bilal-ahmad8/Advanced-Monitoring-System.mlflow/#/experiments/0/runs/2f3749b79b6440a78f344cd3f1181eed
🧪 View experiment at: https://dagshub.com/Bilal-ahmad8/Advanced-Monitoring-System.mlflow/#/experiments/0


[I 2025-06-23 09:51:25,615] Trial 2 finished with value: 0.017145607387647033 and parameters: {'epochs': 40, 'lr': 0.00022991742492107936, 'weight_decay': 0.024057517461496463, 'hidden_dim': 192, 'num_layers': 2, 'biDirect_bool': False, 'dp_ratio_1': 0.2, 'dp_ratio_2': 0.0, 'inside_dp_ratio': 0.5, 'latent_dim': 24, 'batch_size': 16}. Best is trial 2 with value: 0.017145607387647033.
[I 2025-06-23 09:51:46,213] Trial 3 finished with value: 0.026242448017001152 and parameters: {'epochs': 100, 'lr': 0.014764818957477053, 'weight_decay': 0.011897816481599738, 'hidden_dim': 32, 'num_layers': 2, 'biDirect_bool': True, 'dp_ratio_1': 0.5, 'dp_ratio_2': 0.1, 'inside_dp_ratio': 0.1, 'latent_dim': 32, 'batch_size': 8}. Best is trial 2 with value: 0.017145607387647033.


...

[I 2025-06-23 10:01:22,509] Trial 56 finished with value: 0.012777445605024695 and parameters: {'epochs': 90, 'lr': 0.0002774418219145796, 'weight_decay': 0.00010323798038326245, 'hidden_dim': 128, 'num_layers': 1, 'biDirect_bool': False, 'dp_ratio_1': 0.30000000000000004, 'dp_ratio_2': 0.4, 'inside_dp_ratio': 0.30000000000000004, 'latent_dim': 16, 'batch_size': 16}. Best is trial 56 with value: 0.012777445605024695.
[I 2025-06-23 10:01:35,913] Trial 57 finished with value: 0.0152214290574193 and parameters: {'epochs': 90, 'lr': 0.0002850342704918984, 'weight_decay': 0.0001264371781693307, 'hidden_dim': 96, 'num_layers': 1, 'biDirect_bool': False, 'dp_ratio_1': 0.30000000000000004, 'dp_ratio_2': 0.30000000000000004, 'inside_dp_ratio': 0.30000000000000004, 'latent_dim': 16, 'batch_size': 8}. Best is trial 56 with value: 0.012777445605024695.


🏃 View run puzzled-robin-11 at: https://dagshub.com/Bilal-ahmad8/Advanced-Monitoring-System.mlflow/#/experiments/0/runs/044e46742e8545b2a42c7bcf27e66ba1
🧪 View experiment at: https://dagshub.com/Bilal-ahmad8/Advanced-Monitoring-System.mlflow/#/experiments/0


[I 2025-06-23 10:01:42,780] Trial 58 finished with value: 0.013447078177705407 and parameters: {'epochs': 90, 'lr': 0.0002703253587745674, 'weight_decay': 0.0001909177862177531, 'hidden_dim': 128, 'num_layers': 1, 'biDirect_bool': False, 'dp_ratio_1': 0.30000000000000004, 'dp_ratio_2': 0.4, 'inside_dp_ratio': 0.4, 'latent_dim': 16, 'batch_size': 16}. Best is trial 56 with value: 0.012777445605024695.


🏃 View run calm-owl-317 at: https://dagshub.com/Bilal-ahmad8/Advanced-Monitoring-System.mlflow/#/experiments/0/runs/3155a03c164e4d1c8278a496762a7180
🧪 View experiment at: https://dagshub.com/Bilal-ahmad8/Advanced-Monitoring-System.mlflow/#/experiments/0
🏃 View run traveling-stork-446 at: https://dagshub.com/Bilal-ahmad8/Advanced-Monitoring-System.mlflow/#/experiments/0/runs/d1baa2a07a7b4aa5a9720e779be596ee
🧪 View experiment at: https://dagshub.com/Bilal-ahmad8/Advanced-Monitoring-System.mlflow/#/experiments/0


[I 2025-06-23 10:01:52,507] Trial 59 finished with value: 0.01458319928497076 and parameters: {'epochs': 90, 'lr': 0.0007015805508826385, 'weight_decay': 0.0001961305665201039, 'hidden_dim': 128, 'num_layers': 1, 'biDirect_bool': False, 'dp_ratio_1': 0.2, 'dp_ratio_2': 0.4, 'inside_dp_ratio': 0.4, 'latent_dim': 16, 'batch_size': 16}. Best is trial 56 with value: 0.012777445605024695.


In [ ]:
print(study.best_params)
print(study.best_value)

{'epochs': 90, 'lr': 0.0002774418219145796, 'weight_decay': 0.00010323798038326245, 'hidden_dim': 128, 'num_layers': 1, 'biDirect_bool': False, 'dp_ratio_1': 0.30000000000000004, 'dp_ratio_2': 0.4, 'inside_dp_ratio': 0.30000000000000004, 'latent_dim': 16, 'batch_size': 16}
0.012777445605024695


In [16]:
epochs = 90
lr = 0.0002774418219145796
weight_decay = 0.00010323798038326245
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=False,pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False, pin_memory=True)

model = LSTMAutoEncoder(input_dim=6, hidden_dim=128, num_layers=1, biDirect_bool=False, dp_ratio_1=0.30, dp_ratio_2=0.30, latent_dim=16, inside_dp_ratio=0.0).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
criterion = nn.MSELoss()

for i in range(epochs):
  train_loss = train(model, train_loader, batch_size=batch_size, optimizer=optimizer, criterion=criterion)
  val_loss = val(model, val_loader, batch_size=batch_size, optimizer=optimizer, criterion=criterion)
  print(f'Epoch: {i+1}, Train Loss: {train_loss}, Val Loss: {val_loss}')


Epoch: 1, Train Loss: 0.06887767720036209, Val Loss: 0.028579071629792452
Epoch: 2, Train Loss: 0.05362594820326194, Val Loss: 0.019568184856325388
Epoch: 3, Train Loss: 0.04197114199632779, Val Loss: 0.021005841437727213
Epoch: 4, Train Loss: 0.03964998706942424, Val Loss: 0.018699633656069636
Epoch: 5, Train Loss: 0.03838607395300642, Val Loss: 0.018827762454748154
Epoch: 6, Train Loss: 0.03804721403867006, Val Loss: 0.019353498937562108
Epoch: 7, Train Loss: 0.03763783519389108, Val Loss: 0.018953718012198806
Epoch: 8, Train Loss: 0.03703778370982036, Val Loss: 0.018964391900226474
Epoch: 9, Train Loss: 0.03673033963423222, Val Loss: 0.01909711677581072
Epoch: 10, Train Loss: 0.036064094805624336, Val Loss: 0.01880137179978192
Epoch: 11, Train Loss: 0.035500498954206705, Val Loss: 0.01875845226459205
Epoch: 12, Train Loss: 0.034039479680359364, Val Loss: 0.018154600635170937
Epoch: 13, Train Loss: 0.03183204255765304, Val Loss: 0.01739744027145207
Epoch: 14, Train Loss: 0.0296556344

In [ ]:
#torch.save(model.state_dict(), 'model_parameters')
#torch.save(model, 'final_model')

In [ ]:
#mlflow.pytorch.save_model(model, "model")
#mlflow.log_artifact("model")


2025/06/23 10:24:55 WARNING mlflow.utils.requirements_utils: Found torch version (2.6.0+cu124) contains a local version label (+cu124). MLflow logged a pip requirement for this package as 'torch==2.6.0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2025/06/23 10:25:03 WARNING mlflow.utils.requirements_utils: Found torch version (2.6.0+cu124) contains a local version label (+cu124). MLflow logged a pip requirement for this package as 'torch==2.6.0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
